# QuantumOverflow - Full code

Overview:
1. Derive Carleman-linearized Lattice Boltzmann SWE equation for given number of grid points & dimensions
2. Encode the resulting CL-LBE matrix in a linear evolution using the forward Euler approximation
3. Solve the system classically by means of matrix inversion

### Carleman linearisation and Lattice Boltzmann formalism 

Define Carleman linearisation applied to the LBM 

In [32]:
import numpy as np
import sys

class LBM_2_Carlemann:
    def __init__(self):
        # Constants for the D1Q3 lattice (1D, 3 velocities)
        self.w = np.array([2/3, 1/6, 1/6])  # Weights for D1Q3 lattice
        self.e = np.array([0, 1, -1])  # Lattice directions: [0, +1, -1]
        self.c_s = 1 / np.sqrt(3)  # Speed of sound for D1Q3 lattice...is this the delta_x\delta_t ??
        self.g = 9.81  # Acceleration due to gravity (m/s^2)
        self.kn = 0.000000001 #Knudsen number, much less than 1 for chapman-enskogg expansion

        # Parameters
        self.tau = 1.0  # Relaxation time
        self.Nx = 5  # Number of grid points...my code for the F matrices makes the kernel die if this number is 81 or higher
        self.L = 10.0  # Length of the domain (in meters)

        # Initialize macroscopic variables: density(height) and velocity field
        self.h = np.ones(self.Nx)  # height field
        self.u = np.zeros(self.Nx)  # Velocity field

        # Initialize distribution functions (f_i), f has 3 directions (D1Q3)
        self.f = np.zeros((self.Nx, 3))  # Distribution functions for D1Q3
        self.feq = np.zeros_like(self.f)  # Equilibrium distribution functions

     #return a 1D array with one non-zero element of value 1 at specified index
    def one_nonzero(self,dim, n):
        array = np.zeros((dim))
        if n>-1 and n<dim:
            array[n] = 1
        return array
    #make the F matrices for the collision matrix for n grid points
    def gen_F(self):
        f1 = np.zeros((3,3))
        f1[0,1] = f1[0,2] = 1
        f1[1,1] = 1/(2*self.c_s) - 1
        f1[1,2] = -1/(2*self.c_s)
        f1[2,1] = -1/(2*self.c_s)
        f1[2,2] = 1/(2*self.c_s) -1
        f1 = (1/(self.tau*self.kn))*f1
        
        f2 = np.zeros((3,9))
        for i in range(9):
            f2[0,i] = -self.g
            f2[1,i]  = self.g
            f2[2,i]  = self.g
        f2[0,4] = f2[0,4] -4 
        f2[0,8] = f2[0,8] -4
        f2[0,5] = f2[0,5] +4 
        f2[0,7] = f2[0,7] +4
        for i in range(2):
            f2[i+1, 4] = f2[i+1, 4] +2
            f2[i+1, 8] = f2[i+1, 8] +2

            f2[i+1, 5] = f2[i+1, 5] -2
            f2[i+1, 7] = f2[i+1, 7] -2
        f2 = (1/(2*self.tau*self.kn*self.c_s**2))*f2

        f3 = np.zeros((3,9))
        f3[0,4] = 2
        f3[0,8] = 2
        f3[0,5] = -2 
        f3[0,7] = -2
        for i in range(2):
            f3[i+1, 4] = -1
            f3[i+1, 8] = -1

            f3[i+1, 5] = 1
            f3[i+1, 7] = 1
        f3 = np.hstack((f3,f3,f3))
        f3 = (1/(self.tau*self.kn))*f3

        #generalise to n grid points...strategy is to stack matrices, not create matrix of matrices I think...?
        n = self.Nx
        Q = len(self.e)
        '''
        F1 = np.zeros((dim,Q, dim*Q))
        F2 = np.zeros((dim,Q, (dim**2)*(Q**2)))
        F3 = np.zeros((dim,Q, (dim**3)*(Q**3)))
        '''
        I = self.one_nonzero(n, 0)
        F1 = np.kron(I, f1)
        F2 = np.kron(np.kron(I,I) , f2)
        F3 = np.kron(np.kron(np.kron(I, I), I), f3)
        

        for i in range(n-1):
            I = self.one_nonzero(n, i+1)
            F1 = np.vstack((F1, np.kron(I, f1)))
            F2 = np.vstack((F2, np.kron(np.kron(I,I) , f2)))
            F3 = np.vstack((F3, np.kron(np.kron(np.kron(I, I), I), f3)))
        
        return F1,F2,F3
        
    #make A matrices for the collision matrix
    def gen_A(self, F1,F2,F3):
        A11 = F1
        A12 = F2
        A13 = F3
        Q= len(self.e)
        n = self.Nx
        dim = n*Q
        I = np.identity(dim)
        A22 = np.kron(F1,I) + np.kron(I, F1)
        #slows down at A22, kernel dies after with Nx = 50
        A23 = np.kron(F2,I) + np.kron(I, F2)
        A33 = np.kron(np.kron(F1,I), I) + np.kron(np.kron(I,F1), I) + np.kron(np.kron(I,I), F1)

        return A11, A12, A13, A22, A23, A33
    #make collision matrix 
    def gen_collision(self, A11, A12, A13, A22, A23, A33):
        C1 = np.vstack((A11,np.zeros((A22.shape[0]+A33.shape[0],A11.shape[1]))))
        C2 = np.vstack((A12,A22,np.zeros((A33.shape[0],A12.shape[1]))))
        C3 = np.vstack((A13, A23, A33))

        Cc = np.hstack((C1,C2,C3))

        return Cc
    
    #make streaming matrix, restricted to NN, 2nd order accuracy...only makes sense if we are considering more than 1 grid point
    def gen_streaming(self):
        Q= len(self.e)
        n = self.Nx
        inv_delta = n/(2*self.L)
        dim = n*Q
        I = np.identity(dim)
        S = np.zeros((dim, dim))
        for i in range(dim):
            #deal with edge case here...periodic or bounce back BC...here I do code for periodic
            if i<Q:
                S[i,i+Q] = inv_delta*self.e[(i%3)-1]
                S[i, (dim-Q)+i] = -inv_delta*self.e[(i%3)-1]
            elif i>dim -Q - 1:
                S[i,dim-i] = inv_delta*self.e[(i%3)-1]
                S[i,i-Q] = -inv_delta*self.e[(i%3)-1]
            else:
                S[i,i+Q] = inv_delta*self.e[(i%3)-1]
                S[i,i-Q] = -inv_delta*self.e[(i%3)-1]

        B11 = S
        B22 = np.kron(S,I) + np.kron(I, S)
        B33 = np.kron(np.kron(S,I), I) + np.kron(np.kron(I,S), I) + np.kron(np.kron(I,I), S)

        C1 = np.vstack((B11,np.zeros((B22.shape[0]+B33.shape[0],B11.shape[1]))))
        C2 = np.vstack((np.zeros((B11.shape[0],B22.shape[1])),B22,np.zeros((B33.shape[0],B22.shape[1]))))
        C3 = np.vstack((np.zeros((B11.shape[0]+B22.shape[0],B33.shape[1])), B33))

        Cs = np.hstack((C1,C2,C3))

        return Cs


Test matrix for correct dimensions

In [33]:
Matrix_C = LBM_2_Carlemann()
F1,F2,F3  = Matrix_C.gen_F()

A11, A12, A13, A22, A23, A33 = Matrix_C.gen_A(F1,F2,F3)
Cc = Matrix_C.gen_collision(A11, A12, A13, A22, A23, A33)
Cs = Matrix_C.gen_streaming()

CL_LBE_Matrix = Cc +Cs
print("This is the dimension of C: \n", CL_LBE_Matrix.shape)
#print collision matrix and intermediate matrices
np.set_printoptions(threshold=sys.maxsize)
print("checking the dimensions of the matrices \n")
print("dim Matrix F1: \n", F1.shape)
print("dim Matrix F2: \n", F2.shape)
print("dim Matrix F3: \n", F3.shape)
print(" \n")

This is the dimension of C: 
 (3615, 3615)
checking the dimensions of the matrices 

dim Matrix F1: 
 (15, 15)
dim Matrix F2: 
 (15, 225)
dim Matrix F3: 
 (15, 3375)
 



### Map the CL-LBE (ODE) to a LSE via the forward Euler approx.

Embed (ODE) matrix in LSE by "rolling out" the time steps

In [34]:
def embed_matrix(C, delta_t, num_steps):
    """
    Embeds a matrix C into a larger matrix A with specified properties.

    Parameters:
        C (np.ndarray): The matrix to embed (must be square).
        delta_t (float): The length of each time step.
        num_steps (int): The number of time steps (and thus of cascaded blocks in A).

    Returns:
        np.ndarray: The constructed matrix A.
    """
    # Validate inputs
    if not (isinstance(C, np.ndarray) and C.ndim == 2 and C.shape[0] == C.shape[1]):
        raise ValueError("C must be a square matrix.")

    # Identity matrix with the same size as C
    Id = np.eye(C.shape[0])

    # Compute -O = -(Id + delta_t * C)
    O = -(Id + delta_t * C)

    # Size of the large matrix A
    A_size = num_steps * C.shape[0]

    # Initialize A as a zero matrix
    A = np.zeros((A_size, A_size))

    # Fill in the diagonal blocks
    for i in range(num_steps):
        # Main diagonal (Identity blocks)
        start_idx = i * C.shape[0]
        A[start_idx:start_idx + C.shape[0], start_idx:start_idx + C.shape[0]] = Id

        # Secondary diagonal (-O blocks)
        if i > 0:
            prev_idx = (i - 1) * C.shape[0]
            A[start_idx:start_idx + C.shape[0], prev_idx:prev_idx + C.shape[0]] = O

    return A

Test demonstration:

In [35]:
C_test = np.array([[2, 1], [0, 3]])
delta_t = 0.1
num_steps = 3
A = embed_matrix(C_test, delta_t, num_steps)
print("Matrix A:")
print(A)

Matrix A:
[[ 1.   0.   0.   0.   0.   0. ]
 [ 0.   1.   0.   0.   0.   0. ]
 [-1.2 -0.1  1.   0.   0.   0. ]
 [-0.  -1.3  0.   1.   0.   0. ]
 [ 0.   0.  -1.2 -0.1  1.   0. ]
 [ 0.   0.  -0.  -1.3  0.   1. ]]


Apply to Carleman matrix from above given a set of paramters:

In [36]:
N_t = 3  # Number of time steps
delta_t = 0.1  # Time step size
Lin_Euler_Matrix = embed_matrix(CL_LBE_Matrix, delta_t, N_t)

### Embed initial state vector in Euler-approx. formalism

Define initial state vector (as you would for a regular LBM system)

In [37]:
initial_distribution = (1,2,3)  # Function of 3 values in the 1D case

Define CL-linearized state vector

In [38]:
f1 = initial_distribution
f2 = np.kron(f1,f1)
f3 = np.kron(f2,f1)

def create_concatenated_vector(N, f1, f2, f3):
    # Repeat f1 3*N times
    repeated_f1 = np.tile(f1, N)
    # Repeat f2 9*N^2 times
    repeated_f2 = np.tile(f2, (N**2))
    # Repeat f3 27*N^3 times
    repeated_f3 = np.tile(f3, (N**3))
    # Concatenate all repeated arrays
    concatenated_vector = np.concatenate((repeated_f1, repeated_f2, repeated_f3))
    return concatenated_vector

N_grid = 5
phi_t0 = create_concatenated_vector(N_grid, f1, f2, f3)

def append_zeros(f, N):
    # Ensure `f` is a NumPy array
    f = np.array(f)

    # Create N zero vectors of the same shape as `f`
    zero_vector = np.zeros_like(f)
    zeros_to_append = np.tile(zero_vector, (N-1,))

    # Concatenate `f` with the appended zeros
    result = np.concatenate([f, zeros_to_append])

    return result

phi = append_zeros(phi_t0, N_t)
print(phi.shape[0])

10845


Check that dimensions of matrix and state vector match

In [40]:

def check_matrix_vector_dimensions(matrix, vector):
    # Check if the matrix is square
    if matrix.shape[0] != matrix.shape[1]:
        raise ValueError("The input matrix is not square (quadratic).")
    
    # Check if the number of rows/columns matches the vector's length
    return matrix.shape[0] == vector.shape[0]


result = check_matrix_vector_dimensions(Lin_Euler_Matrix, phi)
print(Lin_Euler_Matrix.shape[0])
print("Dimensions match check:", result)

10845
Dimensions match check: True


### Solve system classically 

First check for condition number to estimate range of error

In [45]:
from numpy.linalg import cond

kappa = cond(Lin_Euler_Matrix)
print("Condition number:", kappa)

Condition number: 3.9375469654624375e+26


Solve matrix using Conjugate Gradient method